In [3]:
import sqlparse
from sqlparse.sql import Parenthesis, Statement, Identifier
from sqlparse.tokens import DML, Keyword, Whitespace

def count_subqueries(parsed) -> tuple[int, list[str]]:
    subquery_count: int = 0
    subquery_table_list: list[str] = []

    def extract_table_name(tokens):
        table_name = None
        for i, token in enumerate(tokens):
            # syntax is 'FROM' [<Whitespace>]*  <Table Name>  
            if token.ttype is Keyword and token.value.upper() == 'FROM':
                j = i + 1
                while j < len(tokens) and tokens[j].ttype is Whitespace:
                    j += 1
                # ensure next token is Identifier
                if j < len(tokens) and isinstance(tokens[j], Identifier):
                    table_name = tokens[j].get_real_name()
                    break
        return table_name

    # recursively traverse tokens to find subqueries
    def count_recursive(tokens):
        nonlocal subquery_count 
        nonlocal subquery_table_list
        for token in tokens:
            # Parenthesis with a SELECT statement => subquery
            if isinstance(token, Parenthesis):
                if any(t.ttype is DML and t.value.upper() == 'SELECT' for t in token.tokens):
                    subquery_count += 1
                    subquery_table_list.append(extract_table_name(token.tokens))
                    count_recursive(token.tokens)
            # Recursively check within token if children
            elif hasattr(token, 'tokens'):
                count_recursive(token.tokens)

    count_recursive(parsed.tokens)
    return subquery_count, subquery_table_list

def main():
    # query = \
# """
# SELECT * FROM table1 
# WHERE column1 IN 
#  (SELECT column2 FROM table2 
#    WHERE column3 IN 
#     (SELECT column4 FROM table3)
#  );
# """
    query = "SELECT column1 FROM table1 IN (SELECT column2 FROM table2 WHERE column3 = 'value');"

    token: Statement = sqlparse.parse(query)[0]
    subquery_count, subquery_tables = count_subqueries(token)

    print(f'Number of subqueries: {subquery_count}')
    print(f'Tables in subqueries: {subquery_tables}')

if __name__ == "__main__":
    main()


Number of subqueries: 1
Tables in subqueries: ['table2']


In [ ]:
import sqlparse
from sqlparse.sql import Identifier
from sqlparse.tokens import Keyword, Whitespace

def extract_table_name(tokens):
    table_name = None
    for i, token in enumerate(tokens):
        # syntax is 'FROM' [<Whitespace>]*  <Table Name>  
        if token.ttype is Keyword and token.value.upper() == 'FROM':
            j = i + 1
            while j < len(tokens) and tokens[j].ttype is Whitespace:
                j += 1
            # ensure next token is Identifier
            if j < len(tokens) and isinstance(tokens[j], Identifier):
                table_name = tokens[j].get_real_name()
                break
    return table_name

def main():
    query = "SELECT column2 FROM table2 WHERE column3 = 'value';"
    
    # Parse the SQL query
    parsed = sqlparse.parse(query)[0]
    
    # Extract the table name
    table_name = extract_table_name(parsed.tokens)

    if table_name:
        print(f'Table name: {table_name}')
    else:
        print('No table name found.')

if __name__ == "__main__":
    main()
